In [2]:
import os
import json
import pandas as pd
import traceback


In [4]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import openai
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key = api_key, model_name = "gpt-4o", temperature = 0.5)
# higher the value temperature, more creative is the model

C:\Users\aakri\AppData\Local\Temp\ipykernel_26536\3460240618.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key = api_key, model_name = "gpt-4o", temperature = 0.5)


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

C:\Users\aakri\AppData\Local\Temp\ipykernel_26536\4090386649.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"], template = TEMPLATE
)

In [13]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [14]:
## Now, we'll connect our quiz chain and our review chain

In [15]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"], output_variables = ["quiz","review"], verbose = True )

In [16]:
file_path = r"C:\Users\aakri\mcqgen_proj1\data.txt"

In [17]:
with open(file_path, 'r') as file:
    text = file.read()

In [23]:
import json
json.dumps(RESPONSE_JSON)

#to convert python dict to json string

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
number = 5
subject = "HVAC"
tone = "simple"

In [19]:
# setting up token-usage-tracking

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : text,
            "number" : number,
            "subject" : subject,
            "tone" : tone,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\aakri\AppData\Local\Temp\ipykernel_26536\1632088049.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field's abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).

HVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated with respect to temp

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field's abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).

HVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated w

In [37]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1814
Prompt Tokens:1106
Completion Tokens:708
Total Cost:0.01615


In [26]:
response

{'text': 'Heating, ventilation, and air conditioning (HVAC /ˈeɪtʃˌvæk/) is the use of various technologies to control the temperature, humidity, and purity of the air in an enclosed space. Its goal is to provide thermal comfort and acceptable indoor air quality. HVAC system design is a subdiscipline of mechanical engineering, based on the principles of thermodynamics, fluid mechanics, and heat transfer. "Refrigeration" is sometimes added to the field\'s abbreviation as HVAC&R or HVACR, or "ventilation" is dropped, as in HACR (as in the designation of HACR-rated circuit breakers).\n\nHVAC is an important part of residential structures such as single family homes, apartment buildings, hotels, and senior living facilities; medium to large industrial and office buildings such as skyscrapers and hospitals; vehicles such as cars, trains, airplanes, ships and submarines; and in marine environments, where safe and healthy building conditions are regulated with respect to temperature and humidi

In [39]:
quiz

'```json\n{\n  "1": {\n    "mcq": "What does HVAC stand for?",\n    "options": {\n      "a": "Heating, Ventilation, and Air Conditioning",\n      "b": "Heating, Ventilation, and Cooling",\n      "c": "Heat, Vacuum, and Air Conditioning",\n      "d": "Heat, Ventilation, and Air Circulation"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "Which field of engineering is HVAC system design a subdiscipline of?",\n    "options": {\n      "a": "Civil Engineering",\n      "b": "Electrical Engineering",\n      "c": "Mechanical Engineering",\n      "d": "Chemical Engineering"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "In which environments is HVAC important?",\n    "options": {\n      "a": "Residential structures",\n      "b": "Industrial buildings",\n      "c": "Vehicles and marine environments",\n      "d": "All of the above"\n    },\n    "correct": "d"\n  },\n  "4": {\n    "mcq": "What is the main purpose of ventilation in an HVAC system?",\n    "options": {\n      "a": 

In [38]:
quiz=response.get("quiz")

In [40]:
quiz = quiz.strip()

# Remove Markdown code block if present
if quiz.startswith("```json"):
    quiz = quiz[7:]  # remove the ```json
if quiz.endswith("```"):
    quiz = quiz[:-3]  # remove the trailing ```



In [41]:
quiz

'\n{\n  "1": {\n    "mcq": "What does HVAC stand for?",\n    "options": {\n      "a": "Heating, Ventilation, and Air Conditioning",\n      "b": "Heating, Ventilation, and Cooling",\n      "c": "Heat, Vacuum, and Air Conditioning",\n      "d": "Heat, Ventilation, and Air Circulation"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "Which field of engineering is HVAC system design a subdiscipline of?",\n    "options": {\n      "a": "Civil Engineering",\n      "b": "Electrical Engineering",\n      "c": "Mechanical Engineering",\n      "d": "Chemical Engineering"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "In which environments is HVAC important?",\n    "options": {\n      "a": "Residential structures",\n      "b": "Industrial buildings",\n      "c": "Vehicles and marine environments",\n      "d": "All of the above"\n    },\n    "correct": "d"\n  },\n  "4": {\n    "mcq": "What is the main purpose of ventilation in an HVAC system?",\n    "options": {\n      "a": "To inc

In [42]:
quiz = response.get("quiz")

In [44]:
import json
import re

# Strip ```json and ``` from the start and end
clean_quiz = re.sub(r'^```json\s*|\s*```$', '', quiz.strip())

# Now parse it as JSON
quiz_dict = json.loads(clean_quiz)


In [47]:
quiz_table_data = []
for key,value in quiz_dict.items():
    mcq=value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append(
        {
            "MCQ": mcq,
            "Options": options,
            "Correct": correct
        }
    )

In [48]:
quiz_table_data

[{'MCQ': 'What does HVAC stand for?',
  'Options': 'a : Heating, Ventilation, and Air Conditioning | b : Heating, Ventilation, and Cooling | c : Heat, Vacuum, and Air Conditioning | d : Heat, Ventilation, and Air Circulation',
  'Correct': 'a'},
 {'MCQ': 'Which field of engineering is HVAC system design a subdiscipline of?',
  'Options': 'a : Civil Engineering | b : Electrical Engineering | c : Mechanical Engineering | d : Chemical Engineering',
  'Correct': 'c'},
 {'MCQ': 'In which environments is HVAC important?',
  'Options': 'a : Residential structures | b : Industrial buildings | c : Vehicles and marine environments | d : All of the above',
  'Correct': 'd'},
 {'MCQ': 'What is the main purpose of ventilation in an HVAC system?',
  'Options': 'a : To increase indoor temperature | b : To provide high indoor air quality | c : To reduce noise levels | d : To enhance building aesthetics',
  'Correct': 'b'},
 {'MCQ': 'What are the two categories of building ventilation methods?',
  'Opt

In [50]:
quiz=pd.DataFrame(quiz_table_data)

In [51]:
quiz.to_csv("HVAC.csv", index=False)

In [52]:
pd.DataFrame(quiz_table_data)

,MCQ,Options,Correct
0,What does HVAC stand for?,"a : Heating, Ventilation, and Air Conditioning...",a
1,Which field of engineering is HVAC system desi...,a : Civil Engineering | b : Electrical Enginee...,c
2,In which environments is HVAC important?,a : Residential structures | b : Industrial bu...,d
3,What is the main purpose of ventilation in an ...,a : To increase indoor temperature | b : To pr...,b
4,What are the two categories of building ventil...,a : Natural and artificial | b : Mechanical an...,c
